In [14]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def segment_and_enhance_nets(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the color range for the fluorescent marker
    lower_color = np.array([30, 150, 50])  # Adjust these values based on the color
    upper_color = np.array([85, 255, 255])
    
    # Create a mask for the color
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Apply the mask to get the segmented NETs
    segmented_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Convert the segmented image to grayscale
    gray = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
    
    # Enhance shapes using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    enhanced = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    
    return segmented_image, enhanced

# Example usage
image = cv2.imread('data/unlabeled images/Ctr._1h_DFO-Zeitkinetik_2.jpg')
segmented_image, enhanced_shape = segment_and_enhance_nets(image)

# Display the original, segmented, and enhanced images
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 3, 2)
plt.title('Segmented NETs')
plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
plt.subplot(1, 3, 3)
plt.title('Enhanced Shape')
plt.imshow(enhanced_shape, cmap='gray')
plt.show()




ModuleNotFoundError: No module named 'cv2'

In [ ]:
#### Step 2: CNN Training

Next, use the segmented and shape-enhanced images to train the CNN.

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, concatenate
from tensorflow.keras.optimizers import Adam

# Define a function to create a combined model
def create_combined_model(input_shape):
    # Load the pre-trained VGG16 model for color features
    base_model_color = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    color_output = base_model_color.output
    color_output = GlobalAveragePooling2D()(color_output)
    
    # Separate input for shape-enhanced images
    shape_input = Input(shape=input_shape)
    base_model_shape = VGG16(weights='imagenet', include_top=False, input_tensor=shape_input)
    shape_output = base_model_shape.output
    shape_output = GlobalAveragePooling2D()(shape_output)
    
    # Concatenate color and shape outputs
    combined = concatenate([color_output, shape_output])
    
    # Add custom layers on top of the combined outputs
    x = Dense(1024, activation='relu')(combined)
    predictions = Dense(1, activation='linear')(x)  # For counting, use a single output neuron
    
    # Define the model
    model = Model(inputs=[base_model_color.input, shape_input], outputs=predictions)
    
    # Freeze the layers of the base models
    for layer in base_model_color.layers:
        layer.trainable = False
    for layer in base_model_shape.layers:
        layer.trainable = False
    
    return model

# Create the model
model = create_combined_model((224, 224, 3))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess data
# Assuming you have prepared directories for segmented and enhanced shape images
train_generator_color = datagen.flow_from_directory(
    'path_to_training_data/segmented',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'  # Use 'sparse' if labels are integers (counts)
)

train_generator_shape = datagen.flow_from_directory(
    'path_to_training_data/enhanced_shape',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

validation_generator_color = datagen.flow_from_directory(
    'path_to_validation_data/segmented',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

validation_generator_shape = datagen.flow_from_directory(
    'path_to_validation_data/enhanced_shape',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

# Train the model
model.fit(
    x=[train_generator_color, train_generator_shape],
    epochs=50,
    validation_data=([validation_generator_color, validation_generator_shape])
)

# Save the model
model.save('NET_counting_model_combined.h5')
```